# Packages 

In [35]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import scipy
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
! ls ../data/candidates/ | grep w2v_task2_v2

task2_eval_w2v_task2_v2_True_top100.parquet
task2_test4task3_w2v_task2_v2_True_top100.parquet
task2_test_w2v_task2_v2_True_top100.parquet
task2_train_w2v_task2_v2_True_top100
task2_train_w2v_task2_v2_True_top100.parquet


# Config 

In [37]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [38]:
debug = False


task = 'task2'
version = 'v2'

# # target locales: locales needed for task1
target_locals = ['ES', 'FR', 'IT']

eval_cg = True
train_parts = 20

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None


candidate_path = '../data/candidates/'
model_dir = '../model_training'
ranker_train_data_dir = f'../data/rank_train_data_{task}_{version}'
# train_data_dir = '.'
# test_data_dir = '.'

w2v_model_version = f'w2v_{task}_{version}'
nic_model_version = f'nic_{task}_{version}'
nfi_model_version = f'nfi_{task}_{version}'
covisit_model_version = f'co_visit_{task}_{version}'

# rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
nfi_topn=100
covisit_topn = 300
# PREDS_PER_SESSION = 100


# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")



train_dir = os.path.join(ranker_train_data_dir, 'train')

In [39]:
print(ranker_train_data_dir)
print(train_dir)

../data/rank_train_data_task2_v2
../data/rank_train_data_task2_v2/train


In [40]:
! mkdir {ranker_train_data_dir}

mkdir: cannot create directory ‘../data/rank_train_data_task2_v2’: File exists


In [41]:
! mkdir {train_dir}

mkdir: cannot create directory ‘../data/rank_train_data_task2_v2/train’: File exists


# Function 

In [111]:
def candidates_validation(target_pl):
    # rec_summary  = (target_pl
    #         .lazy()
    #         .select(
    #             pl.col('next_item_prediction').arr.lengths().alias('rec_num')
    #         ).collect().describe())
    # print(rec_summary)
    max_sample_per_NIP = (
        target_pl.groupby(['session_id', 'next_item_prediction'])
            .agg(
                pl.count().alias('unique_count')
                # , pl.col('w2v_weight').max().alias('max_w2v_weight')
                # , pl.col('w2v_weight').min().alias('min_w2v_weight')
            )
            .select(
                pl.col('unique_count').max().alias('max_unique_cnt')
                # , pl.col('max_w2v_weight').min().alias('min_max_w2v')
                # , pl.col('min_w2v_weight').max().alias('max_min_w2v')
            )
    ).to_pandas()['max_unique_cnt'].values[0]
    # print(stat_info)
    # print(max_sample_per_NIP)
    assert max_sample_per_NIP == 1, f"there should be unique prediction for every session_id"

In [118]:



def get_all_candidates(data_type,
                       task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                       , part=None
                      ):
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    print("Candidiate files: ")
    print('+'*20)
    print()
    print(w2v_file)
    print(nic_file)
    print(nfi_file)
    print(covisit_file)
    if data_type == 'train':
        assert part is not None
        w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
        nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
        nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
        covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")
        original_file_name = f"sessions_{data_type}.parquet"
    elif data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    elif data_type == 'test4task3':
        original_file_name = f"sessions_test_task3.parquet"
    elif data_type == 'eval':
        original_file_name = f"sessions_{data_type}.parquet"
    else:
        assert 1==2

        
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


#     def get_vector(prev_items):
#         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
#         return prev_items_vec
    
#     original_pl = (
#             original_pl.with_columns(
#                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
#             )
#         )
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
        pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')
        
    )
    # print(w2v_pl.head().collect())
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    nfi_pl = nfi_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    covisit_pl = covisit_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 
            'w2v_weight'
           ]
    # combined_pl = (
    #     w2v_pl.select(cols)
    #         .join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')
    #         .join(nfi_pl.select(cols), how='left', on='session_id', suffix='_nfi')
    # )

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 
                    'w2v_weight'
                   ]
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols)
                       , nic_pl.select(cols).explode(explode_cols)
                       , nfi_pl.select(cols).explode(explode_cols)
                       , covisit_pl.select(cols).explode(explode_cols)
                      ], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .join(original_pl.select(['session_id', 'prev_items']), how='left', on='session_id')
                .filter(pl.col('prev_items').arr.contains(pl.col('next_item_prediction'))==False)
                .with_columns(
                    pl.col('w2v_weight').max().over('session_id').alias('max_count')
                    , pl.col('w2v_weight').min().over('session_id').alias('min_count')
                )
                .with_columns(
                    pl.when(pl.col('max_count')==pl.col('min_count')).then(1).otherwise((pl.col('w2v_weight')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('w2v_weight')
                )
                .select(
                        pl.exclude(['prev_items', 'max_count', 'min_count'])

                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            .with_columns(
                pl.col('prev_items').arr.lengths().alias('prev_length')
                , pl.col('prev_items').arr.get(-1).alias('last_prev_item')
            ).select(
                pl.all().exclude('prev_items')
            )
    )
    return combined_pl


## Get features

In [110]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type
                , nic_model#=nic_model
                , nfi_model#=nfi_model
                , co_visit_model#=co_visit_model
                , negative_sampling=None, w2v_similarity=False):
    if data_type == 'train':
        # remove data where recall@all>0
        target_df = target_df.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
    target_df = target_df.explode(['next_item_prediction', 'w2v_weight'])

    if data_type not in ['test', 'test4task3']:
        # add target label for the data
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
        sample_pl = target_df.select(cols)
        pos = sample_pl.filter(pl.col('target')==1)
        neg = (sample_pl
                   .filter(pl.col('target')==0)
              )
        neg_original_len = neg.select('session_id').collect().shape[0]
        print(f'Original negative num: {neg_original_len}')
        # neg = (
        #     neg.with_columns(
        #                 pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
        #            )
        #            .filter(pl.col('random')<=negative_sampling)
        #            .select(pl.all().exclude('random'))
        # )
        
        neg = neg.groupby('session_id').agg(
            pl.all().sample(frac=negative_sampling)
            )
        neg = neg.explode(neg.columns[1:])
        # # print(neg.shape)
        # # print(neg.head().collect())
        sample_pl = pl.concat([pos, neg], how='vertical')
        target_df = sample_pl.join(target_df, how='left', on=cols)#.collect()
        del sample_pl
    target_df = (
            target_df.lazy()
                .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
                .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
                .with_columns(
                    # pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    # , pl.when(pl.col('next_item_weight_nfi').is_null()).then(0).otherwise(pl.col('next_item_weight_nfi')).alias('next_item_weight_nfi')
                    # , pl.when(pl.col('next_item_weight_co_visit').is_null()).then(0).otherwise(pl.col('next_item_weight_co_visit')).alias('next_item_weight_co_visit')
            
                    
                    pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    # , pl.struct(["next_item_prediction", "prev_items"]).apply(
                    #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                )
                .with_columns(
                    pl.all().fill_null(0.0)
                )
                .sort('session_id')
                .select(
                    pl.all().exclude('last_prev_item')
                )

    )
    # if w2v_similarity:
    #     target_df = target_df.with_columns(
    #         pl.struct(["next_item_prediction", "prev_items"]).apply(
    #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
    #     )
    return target_df
# target_df.head(3).collect()

In [104]:
# nfi_model.head().collect()

# Load model 

In [17]:
nic_model = (
    pl.scan_parquet(f'../model_training/{nic_model_version}/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight', 'next_item_cnt'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)

nfi_model = (
    pl.scan_parquet(f'../model_training/{nfi_model_version}/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight', 'next_item_normalized_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(nfi_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)


co_visit_model = (
    pl.scan_parquet(f'../model_training/{covisit_model_version}/{covisit_model_version}_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'
                  , 'discount_weight' # co-visit discount weight
         , 'following_weight' 
         , 'following_discount_weight'
            ,'previous_weight'
            , 'previous_discount_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(co_visit_model.schema)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'next_item_cnt': UInt32, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'next_item_normalized_weight': Float64, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'discount_weight': Float32, 'following_weight': Float32, 'following_discount_weight': Float32, 'previous_weight': Float32, 'previous_discount_weight': Float32, 'last_prev_item': Utf8}


# Re-process data for memmory 

In [18]:
def re_process_data(task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=train_parts
                      ):
    parts = train_parts

    data_type = 'train'
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    dir_lst = []
    pl_lst = []
    for file_name in [w2v_file, nic_file, nfi_file, covisit_file]:
        data_pl = pl.scan_parquet(file_name, n_rows=SAMPLE_NUM).collect().sort('session_id')
        dir_name = file_name.replace('.parquet', '')
        print(dir_name)
        dir_lst.append(dir_name)
        pl_lst.append(data_pl)
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
    assert pl_lst[0].shape[0] == pl_lst[1].shape[0] == pl_lst[2].shape[0] == pl_lst[3].shape[0], f"{[ele.shape for ele in pl_lst]}"
    total_len = pl_lst[0].shape[0]

    rows_per_part = int(total_len/parts)

    for dir_path, data_pl in zip(dir_lst, pl_lst):
        print(dir_path)
        for idx in tqdm(range(parts)):
            begin = idx*rows_per_part
            if idx != parts-1:
                end = (idx+1)*rows_per_part
            else:
                end = total_len
            part_pl = data_pl[begin:end :]
            file_name = os.path.join(dir_path, f"part_{idx+1}.parquet")
            print(f'{begin} - {end} -> {file_name}')
            part_pl.write_parquet(file_name)

In [19]:
reprocess_train_data = False

In [20]:
if reprocess_train_data:
    re_process_data()

# Get candiadtes

In [42]:
# data_type = 'eval'
# w2v_file = os.path.join(candidate_path, 
#                            candidate_file_name.format(
#                     task=task
#                 , data_type=data_type
#                 , model_version=w2v_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=w2v_topn
#                            ))
# w2v_file
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)

In [43]:
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


In [44]:
eval_cg_pl = get_all_candidates(data_type='eval')


Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_eval_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_eval_nic_task2_v2_True_top100.parquet
../data/candidates/task2_eval_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_eval_co_visit_task2_v2_True_top300.parquet


In [45]:
# ! ls ../data/candidates/ | grep 'task2'

In [46]:
# ! ls ../data/candidates/ | grep test

In [47]:
test_cg_pl = get_all_candidates(data_type='test')

Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_test_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_test_nic_task2_v2_True_top100.parquet
../data/candidates/task2_test_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_test_co_visit_task2_v2_True_top300.parquet


In [48]:
# eval_cg_pl.head().collect()

In [49]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [50]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [51]:
# ).collect().describe()

In [52]:
# train_cg_pl.head().collect()

In [53]:
# eval_cg_pl.head().collect()

In [33]:
# eval_cg_pl.head().collect()

# Eval candidate generation 

In [78]:
if eval_cg:
    eval_topn = 3000

    col = f"recall@{eval_topn}"
    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    describe = (eval_cg_pl
            .lazy()
            .select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            ).collect().describe())
    print(describe)
    print(final_res)
    del final_res

shape: (7, 2)
┌────────────┬────────────┐
│ describe   ┆ rec_num    │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ count      ┆ 33333.0    │
│ null_count ┆ 0.0        │
│ mean       ┆ 139.811598 │
│ std        ┆ 69.549874  │
│ min        ┆ 1.0        │
│ max        ┆ 428.0      │
│ median     ┆ 122.0      │
└────────────┴────────────┘
shape: (1, 2)
┌────────────────┬─────────────┐
│ total_sessions ┆ recall@3000 │
│ ---            ┆ ---         │
│ u32            ┆ f64         │
╞════════════════╪═════════════╡
│ 33333          ┆ 0.688987    │
└────────────────┴─────────────┘


In [79]:
w2v_model_file

'../model_training/w2v_task2_v2/True.model'

In [80]:
# co_visit_model.head().collect()

In [81]:
# debug_pl.head()

In [82]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train',
#                                        negative_sampling=0.1,
#                                       w2v_similarity=True)
# debug_pl = sampled_train_candidates.head().collect()

In [83]:
# debug_pl

In [89]:
# prev_items = ["B09W9FND7K", "B09JSPLN1M"]
# next_item_prediction = "B09M7GY217"	
# w2vec.wv.n_similarity(prev_items, [next_item_prediction])
# w2vec.wv.n_similarity(prev_items, prev_items)
# prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# next_item_vec = w2vec.wv[next_item_prediction]
# 1 - scipy.spatial.distance.cosine(prev_items_vec, next_item_vec)

# Save Data 

## Save eval data 

In [80]:
# eval_cg_pl

In [85]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [88]:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval'
                             , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                             )

In [89]:
# eval_candidates.head().collect()

In [90]:
%%time
%memit
eval_candidates = eval_candidates.collect()

print('Eval data')
print()
print(eval_candidates.shape)
print(f"Memory: {eval_candidates.estimated_size(unit='gb')}")
print(eval_candidates['target'].mean())
print('Validating')
candidates_validation(target_pl=eval_candidates)
print()

if not debug:
    eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))

peak memory: 1557.81 MiB, increment: 0.00 MiB
Eval data

(4660340, 18)
Memory: 0.5728986747562885
0.004927966629044233
Validating

CPU times: user 2min 44s, sys: 9.37 s, total: 2min 54s
Wall time: 2min 16s


## test data

In [91]:
test_candidates = get_feature(target_df=test_cg_pl, data_type='test'
                            , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                             )

In [92]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [93]:
ranker_train_data_dir

'../data/rank_train_data_task2_v2'

In [94]:
%%time
%memit

test_candidates = test_candidates.collect()
print('Test data')
print()
print(test_candidates.shape)
print(f"Memory: {test_candidates.estimated_size(unit='gb')}")
print('Validating')
candidates_validation(target_pl=test_candidates)

if not debug:
    # eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
    test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

peak memory: 1877.85 MiB, increment: 0.00 MiB
Test data

(5363538, 16)
Memory: 0.5294739129021764
Validating
CPU times: user 3min 10s, sys: 9.77 s, total: 3min 20s
Wall time: 2min 39s


In [95]:
# test_candidates

In [96]:
# eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
# test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

## Saving test4task3 

In [97]:
test4task3_cg_pl = get_all_candidates(data_type='test4task3'
                                                                 
                                     )

Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_test4task3_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_test4task3_nic_task2_v2_True_top100.parquet
../data/candidates/task2_test4task3_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_test4task3_co_visit_task2_v2_True_top300.parquet


In [98]:
test4task3_candidates = get_feature(target_df=test4task3_cg_pl, data_type='test4task3'
                                   , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model)
test4task3_candidates.collect().write_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'))

## Saving train data 

In [99]:
# train_cg_pl.collect().sample(10)

In [119]:
train_dir

'../data/rank_train_data_task2_v2/train'

In [120]:
! ls {train_dir}

In [121]:
if debug:
    train_parts = 1
for part_index in tqdm(range(train_parts)):
    print()
    file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
    print(file_name)
    train_cg_pl = get_all_candidates(data_type='train', part=part_index+1, 
                                     # original_file_name='../data/raw_data_session_id/sess'
                                    )
    sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
                                           negative_sampling=0.1
                                , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                                          )
    sampled_train_candidates = sampled_train_candidates.collect()
    print('Validating')
    candidates_validation(target_pl=sampled_train_candidates)
    # print('Sampled train data')
    print(sampled_train_candidates.shape)
    print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
    print(sampled_train_candidates['target'].mean())
    
    stat_info = (
            sampled_train_candidates
                .groupby('session_id')
                .agg(
                    pl.count().alias('rec_cnt')
                    , pl.col('target').sum().alias('target_num')
                )
                .describe()
        )
    print(stat_info)
    if not debug:
        sampled_train_candidates.write_parquet(file_name)
    # del sampled_train_candidates

  0%|          | 0/20 [00:00<?, ?it/s]


../data/rank_train_data_task2_v2/train/part_1.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_train_nic_task2_v2_True_top100.parquet
../data/candidates/task2_train_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_train_co_visit_task2_v2_True_top300.parquet
Original negative num: 751134
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' ha

  5%|▌         | 1/20 [00:15<04:55, 15.57s/it]

Validating
(78130, 18)
Memory: 0.009602774865925312
0.06513503135799309
shape: (7, 4)
┌────────────┬─────────────┬───────────┬────────────┐
│ describe   ┆ session_id  ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---         ┆ ---       ┆ ---        │
│ str        ┆ f64         ┆ f64       ┆ f64        │
╞════════════╪═════════════╪═══════════╪════════════╡
│ count      ┆ 5089.0      ┆ 5089.0    ┆ 5089.0     │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 3.2810e6    ┆ 15.352722 ┆ 1.0        │
│ std        ┆ 4774.248264 ┆ 7.196559  ┆ 0.0        │
│ min        ┆ 3.272717e6  ┆ 2.0       ┆ 1.0        │
│ max        ┆ 3.289307e6  ┆ 42.0      ┆ 1.0        │
│ median     ┆ 3.281043e6  ┆ 13.0      ┆ 1.0        │
└────────────┴─────────────┴───────────┴────────────┘

../data/rank_train_data_task2_v2/train/part_2.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_train_nic_task2_v2_True_top

 10%|█         | 2/20 [00:27<03:57, 13.21s/it]

...

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
Validating
(78266, 18)
Memory: 0.009620380587875843
0.06558403393555311
shape: (7, 4)
┌────────────┬────────────┬───────────┬────────────┐
│ describe   ┆ session_id ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---        ┆ ---       ┆ ---        │
│ str        ┆ f64        ┆ f64       ┆ f64        │
╞════════════╪════════════╪═══════════╪════════════╡
│ count      ┆ 5133.0     ┆ 5133.0    ┆ 5133.0     │
│ null_count ┆ 0.0        ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 3.5649e6   ┆ 15.247613 ┆ 1.0        │
│ std        ┆ 4769.83783 ┆ 6.710212  ┆ 0.0        │
│ min        ┆ 3.556537e6 ┆ 2.0       ┆ 1.0        │
│ max        ┆ 3.573058e6 ┆ 42.0      ┆ 1.0        │
│ median     ┆ 3.564877e6 ┆ 13.0      ┆ 1.0        │
└────────────┴────────────┴───────────┴─────

 95%|█████████▌| 19/20 [03:58<00:12, 12.15s/it]

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
Validating
(77912, 18)
Memory: 0.009576607495546341
0.06568949584146216
shape: (7, 4)
┌────────────┬─────────────┬───────────┬────────────┐
│ describe   ┆ session_id  ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---         ┆ ---       ┆ ---        │
│ str        ┆ f64         ┆ f64       ┆ f64        │
╞════════════╪═════════════╪═══════════╪════════════╡
│ count      ┆ 5118.0      ┆ 5118.0    ┆ 5118.0     │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 3.5814e6    ┆ 15.223134 ┆ 1.0        │
│ std        ┆ 4830.499617 ┆ 6.813539  ┆ 0.0        │
│ min        ┆ 3.57306e6   ┆ 2.0       ┆ 1.0        │
│ max        ┆ 3.589679e6  ┆ 42.0      ┆ 1.0        │
│ median     ┆ 3.581496e6  ┆ 13.0      ┆ 1.0        │
└────────────┴─────────────┴───────────┴────────────┘

../data/rank_train_data_task2_v2/train/part_20.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_ta

100%|██████████| 20/20 [04:14<00:00, 12.72s/it]

object of type 'NoneType' has no len()
Validating
(77263, 18)
Memory: 0.009496822953224182
0.06621539417314885
shape: (7, 4)
┌────────────┬─────────────┬───────────┬────────────┐
│ describe   ┆ session_id  ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---         ┆ ---       ┆ ---        │
│ str        ┆ f64         ┆ f64       ┆ f64        │
╞════════════╪═════════════╪═══════════╪════════════╡
│ count      ┆ 5116.0      ┆ 5116.0    ┆ 5116.0     │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 3.5979e6    ┆ 15.102228 ┆ 1.0        │
│ std        ┆ 4791.837786 ┆ 6.806666  ┆ 0.0        │
│ min        ┆ 3.589681e6  ┆ 2.0       ┆ 1.0        │
│ max        ┆ 3.606247e6  ┆ 43.0      ┆ 1.0        │
│ median     ┆ 3597971.5   ┆ 13.0      ┆ 1.0        │
└────────────┴─────────────┴───────────┴────────────┘


# Slow speed -> **7 hours to finished all of the data**



```
../data/rank_train_data_v2/train/part_1.parquet
Validating
(28566071, 11)
Memory: 2.23475506529212
0.005130737090165463
shape: (7, 4)
┌────────────┬──────────────┬───────────┬────────────┐
│ describe   ┆ session_id   ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---          ┆ ---       ┆ ---        │
│ str        ┆ f64          ┆ f64       ┆ f64        │
╞════════════╪══════════════╪═══════════╪════════════╡
│ count      ┆ 146565.0     ┆ 146565.0  ┆ 146565.0   │
│ null_count ┆ 0.0          ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 81754.567209 ┆ 194.90377 ┆ 1.0        │
│ std        ┆ 47232.319874 ┆ 96.523984 ┆ 0.0        │
│ min        ┆ 0.0          ┆ 80.0      ┆ 1.0        │
│ max        ┆ 163580.0     ┆ 468.0     ┆ 1.0        │
│ median     ┆ 81763.0      ┆ 154.0     ┆ 1.0        │
└────────────┴──────────────┴───────────┴────────────┘
  5%|▌         | 1/20 [22:21<7:04:44, 1341.26s/it]

../data/rank_train_data_v2/train/part_2.parquet

```

# Eval

In [ ]:
# ! ls {ranker_train_data_dir}

In [ ]:
# ! rm {ranker_train_data_dir}/*

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet')).collect().shape

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet')).collect().shape

# Debug 

In [43]:
# part_index = 0
# print()
# file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
# print(file_name)
# train_cg_pl = get_all_candidates(data_type='train', part=part_index+1)
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
#                                        # negative_sampling=0.1
#                                       )
# sampled_train_candidates = sampled_train_candidates.collect()
# print('Validating')
# candidates_validation(target_pl=sampled_train_candidates)
# # print('Sampled train data')
# print(sampled_train_candidates.shape)
# print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
# print(sampled_train_candidates['target'].mean())

# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .describe()
#     )
# print(stat_info)
# (
#         train_cg_pl.select(
#                 pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#             )
#             .collect()
#             .describe(
#                 # percentiles=[0.1*i for i in range(10)]
#             )
#     )


../data/rank_train_data_v2/train/part_1.parquet
Validating
(335696, 11)
Memory: 0.02626188099384308
0.010298007721271627
shape: (7, 4)
┌────────────┬─────────────┬───────────┬────────────┐
│ describe   ┆ session_id  ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---         ┆ ---       ┆ ---        │
│ str        ┆ f64         ┆ f64       ┆ f64        │
╞════════════╪═════════════╪═══════════╪════════════╡
│ count      ┆ 3457.0      ┆ 3457.0    ┆ 3457.0     │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 5540.907723 ┆ 97.106161 ┆ 1.0        │
│ std        ┆ 3186.084851 ┆ 1.80037   ┆ 0.0        │
│ min        ┆ 5.0         ┆ 79.0      ┆ 1.0        │
│ max        ┆ 11052.0     ┆ 100.0     ┆ 1.0        │
│ median     ┆ 5470.0      ┆ 98.0      ┆ 1.0        │
└────────────┴─────────────┴───────────┴────────────┘


describe,rec_num
str,f64
"""count""",10000.0
"""null_count""",0.0
"""mean""",97.7283
"""std""",1.868804
"""min""",79.0
"""max""",100.0
"""median""",98.0


In [45]:
# (train_cg_pl.with_columns(
#         pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#     )
#     .filter(pl.col('rec_num') == 100)
#     .head().collect())

session_id,next_item_prediction,w2v_weight,next_item,locale,prev_length,last_prev_item,rec_num
i64,list[str],list[f64],str,str,u32,str,u32
9184,"[""B08447LBF5"", ""B073VPDRP1"", … ""B08FB3KWXL""]","[0.868687, 0.515152, … 0.828283]","""B017RJW8OW""","""DE""",3,"""B09XF26MW6""",100
5152,"[""B01LZPR94B"", ""B0B6H7NDBS"", … ""B0B9X73PB5""]","[0.787879, 0.515152, … 0.79798]","""B0B8S79JGZ""","""DE""",3,"""B0BJPZHZRB""",100
5920,"[""B09MHXJ4QK"", ""B00C81E9OW"", … ""B0B372LV18""]","[0.393939, 0.585859, … 0.262626]","""B098DY9JS2""","""DE""",2,"""B094TZ4V98""",100
8480,"[""B01E5A7KIK"", ""B094JG3PBD"", … ""B07Z9H23XK""]","[0.181818, 0.373737, … 0.515152]","""B014PI4HGK""","""DE""",2,"""B0B5GZYMJH""",100
1152,"[""B095R6SRG7"", ""B07NY59JGM"", … ""B08BXH5P73""]","[0.353535, 0.464646, … 0.262626]","""B07WPY21TW""","""DE""",2,"""B00FYV34FI""",100


In [65]:
# search_id = 9184

In [48]:
# data_type = 'train'
# part = 0 +1 
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model_version
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# nfi_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=nfi_model_version
#             , model_for_eval=model_for_eval
#             , topn=nfi_topn
#                    ))
# covisit_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=covisit_model_version
#             , model_for_eval=model_for_eval
#             , topn=covisit_topn
#                    ))
# if data_type == 'train':
#     assert part is not None
#     w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
#     covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")

# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


# #     def get_vector(prev_items):
# #         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# #         return prev_items_vec

# #     original_pl = (
# #             original_pl.with_columns(
# #                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
# #             )
# #         )

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
#     pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')

# )
# # print(w2v_pl.head().collect())
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# nfi_pl = nfi_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# covisit_pl = covisit_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# cols = ['session_id', 'next_item_prediction', 
#         'w2v_weight'
#        ]


In [57]:
# nic_rec = nic_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]
# w2v_rec = w2v_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]